In [3]:
import scipy
import os

import basic_mne_functions as bmf

In [4]:
# epoch length
epoch = 1
# wake time (s) to save before first sleep and after last sleep
# (30 mins, so 30(s) * 60(s))
wake_time = 30 * 60
# which channels to extract
channels = ["Fpz-Cz", "Pz-Oz", "horizontal", "submental"]

# path to raw PSG files
raw_path = "C:/Users/andri/school/bio-informatics/internship/donders/data/human_test_data/input/raw_input"
# path to raw hypnogram annotation files
anno_path = "C:/Users/andri/school/bio-informatics/internship/donders/data/human_test_data/input/annotation_raw"
# path to the output for the .mat files
output_path = "C:/Users/andri/school/bio-informatics/internship/donders/data/human_test_data/mat_files/"

### test files (comment when not testing)
# raw_file = "/SC4481F0-PSG.edf"
# anno_file = "/SC4481FV-Hypnogram.edf"

# iterate over both PSG and Hypno directories and get the file paths
for raw_file, anno_file in zip(os.listdir(raw_path), os.listdir(anno_path)):
    # empty list to store sleep stages in 1d array where each 
    # element is 1 epoch
    sleep_states = []
    get_annotation = False
    subject = raw_file.split("/")[-1].split("-")[0]
    # extract raw data
    raw = bmf.read_raw_psg(f"{raw_path}/{raw_file}", channels)
    anno = bmf.extract_annotation(f"{anno_path}/{anno_file}", raw)
    cropped_raw = bmf.crop_data(raw, wake_time)
    for channel in ["Fpz-Cz", "Pz-Oz", "horizontal", "submental"]:
        cropped_raw_c = cropped_raw.copy().pick([channel])
        while get_annotation == False:
            sleep_events = bmf.create_sleep_events(cropped_raw_c, epoch)
            # create array from sleep events with only sleep stages
            for event in sleep_events:
                sleep_states.append(int(event[-1]))
            scipy.io.savemat(
                f"{output_path}{subject}_sleep_states.mat", 
                {"States": sleep_states}
            )
            get_annotation = True
        # rename indistinct channel names
        if channel == "horizontal":
            cropped_raw_c.rename_channels({"horizontal": "EOG"})
            channel = "EOG"
        elif channel == "submental":
            cropped_raw_c.rename_channels({"submental":"EMG"})
            channel = "EMG"
        # save numpy array to .mat file format
        scipy.io.savemat(
            f"{output_path}{subject}_{channel}.mat", 
            {channel: cropped_raw_c._data.reshape((-1,1))}
            )
    

Extracting EDF parameters from C:\Users\andri\school\bio-informatics\internship\donders\data\human_test_data\input\raw_input\SC4001E0-PSG.edf...
EDF file detected
Channel 'EEG Fpz-Cz' recognized as type EEG (renamed to 'Fpz-Cz').
Channel 'EEG Pz-Oz' recognized as type EEG (renamed to 'Pz-Oz').
Channel 'EOG horizontal' recognized as type EOG (renamed to 'horizontal').
Channel 'Resp oro-nasal' recognized as type RESP (renamed to 'oro-nasal').
Channel 'EMG submental' recognized as type EMG (renamed to 'submental').
Channel 'Temp rectal' recognized as type TEMP (renamed to 'rectal').
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7949999  =      0.000 ... 79499.990 secs...


c:\Users\andri\school\bio-informatics\internship\donders\vsc\Human_SleepSCoring\DilonAndriesse\pre_processing\basic_mne_functions.py:15: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(
c:\Users\andri\school\bio-informatics\internship\donders\vsc\Human_SleepSCoring\DilonAndriesse\pre_processing\basic_mne_functions.py:15: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(


First sleep starts at: 30630.0
Last sleep ends at: 52260.0
Cropping raw: 28830.0 - 54060.0
Cropping finished.
Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]
Extracting EDF parameters from C:\Users\andri\school\bio-informatics\internship\donders\data\human_test_data\input\raw_input\SC4002E0-PSG.edf...
EDF file detected
Channel 'EEG Fpz-Cz' recognized as type EEG (renamed to 'Fpz-Cz').
Channel 'EEG Pz-Oz' recognized as type EEG (renamed to 'Pz-Oz').
Channel 'EOG horizontal' recognized as type EOG (renamed to 'horizontal').
Channel 'Resp oro-nasal' recognized as type RESP (renamed to 'oro-nasal').
Channel 'EMG submental' recognized as type EMG (renamed to 'submental').
Channel 'Temp rectal' recognized as type TEMP (renamed to 'rectal').
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8489999  =      0.000 ... 84899.990 

c:\Users\andri\school\bio-informatics\internship\donders\vsc\Human_SleepSCoring\DilonAndriesse\pre_processing\basic_mne_functions.py:15: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(
c:\Users\andri\school\bio-informatics\internship\donders\vsc\Human_SleepSCoring\DilonAndriesse\pre_processing\basic_mne_functions.py:15: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(


First sleep starts at: 26070.0
Last sleep ends at: 56310.0
Cropping raw: 24270.0 - 58110.0
Cropping finished.
Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]
Extracting EDF parameters from C:\Users\andri\school\bio-informatics\internship\donders\data\human_test_data\input\raw_input\SC4011E0-PSG.edf...
EDF file detected
Channel 'EEG Fpz-Cz' recognized as type EEG (renamed to 'Fpz-Cz').
Channel 'EEG Pz-Oz' recognized as type EEG (renamed to 'Pz-Oz').
Channel 'EOG horizontal' recognized as type EOG (renamed to 'horizontal').
Channel 'Resp oro-nasal' recognized as type RESP (renamed to 'oro-nasal').
Channel 'EMG submental' recognized as type EMG (renamed to 'submental').
Channel 'Temp rectal' recognized as type TEMP (renamed to 'rectal').
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8405999  =      0.000 ... 84059.990 

...

First sleep starts at: 32430.0
Last sleep ends at: 64380.0
Cropping raw: 30630.0 - 66180.0
Cropping finished.
Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]
Extracting EDF parameters from C:\Users\andri\school\bio-informatics\internship\donders\data\human_test_data\input\raw_input\SC4821G0-PSG.edf...
EDF file detected
Channel 'EEG Fpz-Cz' recognized as type EEG (renamed to 'Fpz-Cz').
Channel 'EEG Pz-Oz' recognized as type EEG (renamed to 'Pz-Oz').
Channel 'EOG horizontal' recognized as type EOG (renamed to 'horizontal').
Channel 'Resp oro-nasal' recognized as type RESP (renamed to 'oro-nasal').
Channel 'EMG submental' recognized as type EMG (renamed to 'submental').
Channel 'Temp rectal' recognized as type TEMP (renamed to 'rectal').
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8165999  =      0.000 ... 81659.990 

c:\Users\andri\school\bio-informatics\internship\donders\vsc\Human_SleepSCoring\DilonAndriesse\pre_processing\basic_mne_functions.py:15: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(
c:\Users\andri\school\bio-informatics\internship\donders\vsc\Human_SleepSCoring\DilonAndriesse\pre_processing\basic_mne_functions.py:15: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(


First sleep starts at: 11070.0
Last sleep ends at: 58620.0
Cropping raw: 9270.0 - 60420.0
Cropping finished.
Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]
Extracting EDF parameters from C:\Users\andri\school\bio-informatics\internship\donders\data\human_test_data\input\raw_input\SC4822G0-PSG.edf...
EDF file detected
Channel 'EEG Fpz-Cz' recognized as type EEG (renamed to 'Fpz-Cz').
Channel 'EEG Pz-Oz' recognized as type EEG (renamed to 'Pz-Oz').
Channel 'EOG horizontal' recognized as type EOG (renamed to 'horizontal').
Channel 'Resp oro-nasal' recognized as type RESP (renamed to 'oro-nasal').
Channel 'EMG submental' recognized as type EMG (renamed to 'submental').
Channel 'Temp rectal' recognized as type TEMP (renamed to 'rectal').
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8429999  =      0.000 ... 84299.990 s

c:\Users\andri\school\bio-informatics\internship\donders\vsc\Human_SleepSCoring\DilonAndriesse\pre_processing\basic_mne_functions.py:15: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(
c:\Users\andri\school\bio-informatics\internship\donders\vsc\Human_SleepSCoring\DilonAndriesse\pre_processing\basic_mne_functions.py:15: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(


First sleep starts at: 24120.0
Last sleep ends at: 61500.0
Cropping raw: 22320.0 - 63300.0
Cropping finished.
Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]
